# DB 連線

In [6]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd, numpy as np
import matplotlib.pyplot as plt

import json, zipfile, io, os, re, datetime, requests, adodbapi

In [7]:
conn = adodbapi.connect("PROVIDER=SQLOLEDB;Data Source={0};Initial Catalog={1};Persist Security Info=False;User ID={2};Password={3};App=CMReport"
               .format("203.74.190.157,443","northwind","xRsu874pw3","dkem!ygh$746h"))

# 個股即時新聞

In [3]:
sql = "select top 10 日期,代號,名稱,新聞標題,新聞內容 from sysnews with(nolock)"
data = pd.read_sql(sql, conn)
data

,日期,代號,名稱,新聞標題,新聞內容
0,20210326,1477,聚陽,和碩、聚陽 認購搶鏡,台股多空交戰，盤面個股各自表現，和碩（4938）有轉型契機；聚陽（1477）則有獲利回溫空間...
1,20210326,4938,和碩,和碩、聚陽 認購搶鏡,台股多空交戰，盤面個股各自表現，和碩（4938）有轉型契機；聚陽（1477）則有獲利回溫空間...
2,20210326,2454,聯發科,聯詠、聯陽 瞄準長天期,台股昨（25）日再度面臨「萬六」保衛戰，早盤回測整數關卡後，多方進場拉抬，又以IC設計族群表...
3,20210326,1907,永豐餘,永豐餘 挑價外15%,由於消費習慣改變，電商網購崛起，加上台商持續回流，帶動台灣工紙需求增加，激勵昨（25）日台股...
4,20210326,2855,統一證,永豐餘 挑價外15%,由於消費習慣改變，電商網購崛起，加上台商持續回流，帶動台灣工紙需求增加，激勵昨（25）日台股...
5,20210326,6138,茂達,茂達 選逾三個月,風扇驅動IC與電源管理IC茂達（6138），去年賺進超過半個股本，今年展望也樂觀，昨（25）...
6,20210326,2454,聯發科,聯發科 二檔耀眼,IC設計龍頭股聯發科（2454）昨（25）日重返900元關卡，進而帶動加權指數由黑翻紅。聯發...
7,20210327,1402,遠東新,終止連九賣 外資反手大買台積,"外資終止對台積電（2330）的連九賣、昨（26）日反手買超6,859張，帶領記憶體、IC設計..."
8,20210327,2303,聯電,終止連九賣 外資反手大買台積,"外資終止對台積電（2330）的連九賣、昨（26）日反手買超6,859張，帶領記憶體、IC設計..."
9,20210327,2330,台積電,終止連九賣 外資反手大買台積,"外資終止對台積電（2330）的連九賣、昨（26）日反手買超6,859張，帶領記憶體、IC設計..."


In [ ]:
cnt=dict()
for i in list(data['名稱']):
    if i in cnt:
        cnt[i]+=1
    else:
        cnt[i]=1
cnt=[[i,cnt[i]] for i in cnt]
cnt.sort(key=lambda x:-x[1])
for name,c in cnt:
    sql = f"select top 1 代號 from sysnews with(nolock) where 名稱='{name}'"
    data = pd.read_sql(sql, conn)
    if not re.fullmatch(r'\d{4}',data['代號'][0]):
        print(name,c,data['代號'][0])

# 個股新聞力評分
多空強度：[-100,100]
前10大

In [4]:
sql = "SELECT top 10 * FROM sysnearp with(nolock)"
df = pd.read_sql(sql, conn)
df

,日期,股票代號,股票名稱,新聞則數,平均新聞則數,曝光強度(%),多空強度(%),新聞力評分(%),收盤漲跌幅(%),相對大盤漲跌幅(%),成交量漲跌幅(%),備註來源,RTIME
0,20030217,6214,精誠,5,1.50,94,56,88,1.54,-3.16,12.420000,"鉅亨網*1,工商*4",None
1,20030214,6214,精誠,7,1.26,100,52,87,-2.26,-1.95,-59.700001,"鉅亨網*4,中央社*1,智富網*2",None
2,20030213,6214,精誠,2,1.23,65,22,31,5.56,8.09,127.900002,"鉅亨網*1,智富網*1",None
3,20030212,6214,精誠,1,1.24,45,0,-33,-3.08,-3.21,31.990000,智富網*1,None
4,20030211,6214,精誠,0,1.30,26,0,-34,1.56,2.10,-34.160000,None,None
5,20030210,6214,精誠,1,1.32,44,6,-19,-5.19,-3.26,-62.130001,鉅亨網*1,None
6,20030207,6214,精誠,0,1.39,26,0,-35,-0.74,1.29,0.400000,None,None
7,20030206,6214,精誠,8,1.00,100,40,66,-4.23,-0.61,67.529999,"鉅亨網*4,工商*2,中央社*2",None
8,20030128,6214,精誠,4,0.81,100,80,100,6.77,5.91,36.250000,鉅亨網*4,None
9,20030127,6214,精誠,0,0.87,24,0,-25,6.40,8.08,36.189999,None,None


In [37]:
sql = "SELECT top 10 [新聞力評分(%)],[相對大盤漲跌幅(%)],[多空強度(%)],日期,股票代號 FROM sysnearp with(nolock) where [多空強度(%)]!=0"
df = pd.read_sql(sql, conn)
df

,新聞力評分(%),相對大盤漲跌幅(%),多空強度(%),日期,股票代號
0,-100,-0.18,-24,20210312,1216
1,-100,-1.07,-33,20210312,1258
2,-97,0.18,-20,20210312,1303
3,-100,0.09,-33,20210312,1326
4,100,2.56,19,20210312,1336
5,100,1.95,100,20210312,1432
6,-100,-0.31,-50,20210312,1434
7,-65,0.73,-3,20170509,8069
8,100,-3.31,86,20170509,8077
9,-97,-0.56,-25,20170509,8080


In [ ]:
indexSET=([df['日期'][i]+df['股票代號'][i] for i in range(len(df))])
print(len(indexSET))

# MyCode...
```json
[
    {
        "stock_id": 2330,
        "stock_name": "台積電",
        "news": [
            {
                "date": "20210225",
                "news": [
                    {"title": "this is a news title", "content": "this is the news content"},
                    {"title": "this is a news title", "content": "this is the news content"},
                    ...
                ],
                "index": [...(different kinds of indices)]
            }, ...
        ]
    }, ...
]
```

In [ ]:
rank='2330 3037 2303 6488 8046 2603 3481 2458 2327 2454 2609 3006 2409 2317 2344 3443 2492 3260 2337 6533 3034 4976 3008 5483 4743 2408 2308 6116 3189 2354 2357 6531 3711 2014 4142 2377 3105 3228 5269 2375 8070 3661 4968 3545 3078 3406 5471 8299 1301 6271 2331 1305 2338 2881 3540 2882 3019 6202 4967 3042 4128 4938 2610 2618 0050 2328 2379 3413 2376 2368 3374 2371 2002 2891 8086 3293 2356 6138 2615 4966 2027 6150 2353 3707 6121 1590 2886 6415 6203 3576 2049 1326 4958 4162 8358 5223 6237 8936 2412 2383 6525 6269 2382 2449 2201 3680 3178 3014 8069 5347 3714 8081 2481 1101 1513 6213 2472 6443 6477 6182 5425 2345 2009 1314 2301 5371 3691 1303 4915 5274 2312 1909 3141 3529 8383 2884 6188 6669 6541 3552 2467 6679 3015 2038 6505 8454 2108 5871 3533 2342 3665 3264 5243 3317 6510 1216 1476 2401 6224 3231 6104 1309 8016 8464 2888 6278 2456 0056 1477 5386 2474 3419 4576 2912 2436 6223 2883 6239 1312 6274 3483 5299 6414 8255 9958 6285 5469 8171 2352 2885 2476 1785 2441 6147 2406 6214'.split(' ')
sql = "SELECT [新聞力評分(%)],[相對大盤漲跌幅(%)],[多空強度(%)],日期,股票代號 FROM sysnearp with(nolock)"
idxdf = pd.read_sql(sql, conn)
idxdf = idxdf.where(idxdf.notnull(),None)
index=dict()
for i in tqdm(range(len(idxdf))):
    index[f"{idxdf['日期'][i]} {idxdf['股票代號'][i]}"]=[
        idxdf['新聞力評分(%)'][i],
        idxdf['相對大盤漲跌幅(%)'][i],
        idxdf['多空強度(%)'][i]
    ]
def get_index(sid,date):
    return index[f'{date} {sid}'] if f'{date} {sid}' in index else [None]*3
def get_news(data,stock_id):
    res=[]
    for i in range(len(data)):
        if not res or res[-1]['date']!=data['日期'][i]:
            res.append({'date': data['日期'][i], 'news': [], 'index': get_index(stock_id,data['日期'][i])})
        res[-1]['news'].append({'title': data['新聞標題'][i], 'content': data['新聞內容'][i]})
    return res
res=[]
for i in tqdm(rank):
    sql = f"select * from sysnews with(nolock) where 代號 = '{i}' order by '發布日期時間' ASC"
    data = pd.read_sql(sql, conn)
    res.append({
        "stock_id": i,
        "stock_name": data['名稱'][0],
        "news": get_news(data,i)
    })

 86%|██████████████████████████████████████████████████████████████████████████████▌            | 7046210/8155454 [1:31:11<133:03:13,  2.32it/s]

In [ ]:
class Encoder(json.JSONEncoder):
    def default(self,obj):
        if isinstance(obj,np.int64):
            return int(obj)
        return json.JSONEncoder.default(self,obj)
with open('./data/collection.json','w+',encoding='utf-8') as f:
    json.dump(res,f,indent=2,ensure_ascii=False,cls=Encoder)

# filter

In [ ]:
import json
with open('./data/collection.json','r',encoding='utf-8') as f:
    data=json.load(f)

In [ ]:
import re
import numpy as np
def non_empty(a):
    return list(filter(len,a))
filters=[
    ('title', '公告'),
    ('content', '\n\d+\.[^\d]'),
    ('content', '事實發生日'),
    ('content', '\n[一二三四五六七八九十]、'),
    ('content', '\n\s{8}')
]
count=np.zeros((len(filters)+1,))
trash=[]
def clean(da):
    global count, filters
    trash.append((da['title'],da['content']))
    for i,(ctx,regex) in enumerate(filters):
        if re.search(regex, da[ctx].strip()):
            count[i] += 1
            trash[-1] += (i,)
            return False
    count[-1] += 1
    trash.pop()
    return True
def cleanse(data):
    for d in data:
        for n in d['news']:
            n['news']=list(filter(clean,n['news']))
        d['news']=list(filter(lambda x: len(x['news']),d['news']))
cleanse(data)
print(count)
print(100 * count / count.sum())
print('ok: {:3.2%}'.format(count[-1] / count.sum()))

In [ ]:
with open('./data/clean.json','w+',encoding='utf-8') as f:
    json.dump(data,f,indent=2,ensure_ascii=False)

In [3]:
with open('./data/clean.json','r',encoding='utf-8') as f:
    data=json.load(f)
newsSET=[]
news=[]
for i in data:
    for j in i['news']:
        newsSET.append(j['date']+i['stock_id'])
        for k in j['news']:
            news.append((k['title'],k['content']))
print(len(newsSET))

25714


In [6]:
np.array(list(map(lambda x : len(x[0]),news))).max()

93

In [ ]:
np.random.shuffle(news)
for [a,b] in news[:20]:
    print('-'*50)
    print('--', a)
    print(b)

In [ ]:
print(len(newsSET))
print(len(indexSET))

In [ ]:
print(len(set(newsSET)-set(indexSET)))
print(len(set(indexSET)-set(newsSET)))